# Friday 3.2
let's try this again but rip off ChatGPT

## Imports

In [2]:
import re
from transformers import AutoTokenizer, AutoModelForCausalLM,  AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import torch

2023-01-04 21:33:23.241019: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-04 21:33:23.241054: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Load base model

In [3]:
# Set the model and tokenizer names
model_name = "bigscience/T0_3B"
tokenizer_name = "bigscience/T0_3B"

# Load the model and tokenizer
# model =  AutoModelForSeq2SeqLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

## Level fine tunning
Do a statndard fine tune on the model

### The finetune function

In [4]:
def fine_tune(model, train_dataloader, val_dataloader, epochs=2):
  # Set up the trainer
  training_args = TrainingArguments(
      output_dir="./results",          # output directory
      num_train_epochs=epochs,         # number of epochs
      per_device_train_batch_size=16,  # batch size
      per_device_eval_batch_size=64,   # evaluation batch size
      warmup_steps=500,                # number of warmup steps
      weight_decay=0.01,               # L2 weight decay
      logging_dir="./logs"             # logging directory
  )

  trainer = Trainer(
      model=model,                     # the model to train
      args=training_args,              # training arguments
      train_dataset=train_dataloader,  # training dataset
      eval_dataset=val_dataloader      # validation dataset
  )

  # Fine-tune the model
  trainer.train()

### Data loader

In [6]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
dataset = load_dataset("Anthropic/hh-rlhf")
def tokenize_function(examples):
    return tokenizer(examples["chosen"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Using pad_token, but it is not set yet.
Using custom data configuration Anthropic--hh-rlhf-03696fd7b9f56a9e
Reusing dataset json (/home/vscode/.cache/huggingface/datasets/Anthropic___json/Anthropic--hh-rlhf-03696fd7b9f56a9e/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/161 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

In [7]:
train_dataloader = tokenized_datasets["train"].shuffle(seed=42)
val_dataloader = tokenized_datasets["test"].shuffle(seed=42)

### Tune it!

In [8]:
fine_tune(model, train_dataloader, val_dataloader)

The following columns in the training set don't have a corresponding argument in `GPTNeoForCausalLM.forward` and have been ignored: rejected, chosen. If rejected, chosen are not expected by `GPTNeoForCausalLM.forward`,  you can safely ignore this message.
/home/vscode/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 160800
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 20100


IndexError: index out of range in self

In [ ]:
def generate_response_single(prompt):
  # Encode the prompt and generate a response
  input_ids = tokenizer.encode(prompt, return_tensors='pt')
  response = model.generate(input_ids, max_length=1024, top_p=0.9, top_k=20)
generate_response_single("Human: Hello there Assistant: ")

## Chatbot interface

In [ ]:
def generate_response(prompt, chat_history=[]):
  # Encode the prompt and generate a response
  # input_ids = tokenizer.encode(prompt + tokenizer.eos_token, return_tensors='pt')
  input_ids = torch.tensor(tokenizer.encode(' '.join(chat_history) + tokenizer.eos_token + prompt, return_tensors='pt'))
  response = model.generate(input_ids, max_length=1024, pad_token_id=tokenizer.eos_token_id)

  # Decode the response and return it
  return tokenizer.decode(response[0], skip_special_tokens=True)

# Test the chatbot with a prompt
prompt = "Hello there, how are you?"
response = generate_response(prompt)
print(response)


def chatbot(prompt):
  """Chatbot main loop. Prompt the user for input and generate a response."""
  chat_history = []
  while True:
    # Get the user's input
    user_input = input(prompt)
    if user_input.lower() == 'exit':
      print("Goodbye!")
      break

    # Add the user's input to the chat history
    chat_history.append(user_input)

    # Generate a response
    response = generate_response(user_input, chat_history)
    print(response)

    # Add the chatbot's response to the chat history
    chat_history.append(response)
chatbot("you>>")

/tmp/ipykernel_2740505/335509339.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(tokenizer.encode(' '.join(chat_history) + tokenizer.eos_token + prompt, return_tensors='pt'))


I am a student at the University of Southern California.
There is a problem with the server.
There is a problem with the server.
There is a problem with the server.


KeyboardInterrupt: Interrupted by user

In [ ]:
def chatbot():
  step = 0
  chat_history = []
  while(True):
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("Friday: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
    step = step + 1

# Run the chatbot
chatbot()

Friday: 
Friday: 
Friday: 
Friday: 


KeyboardInterrupt: 